In [1]:
import os
import shutil 
# 這兩個套件是為了要將訓練資料的路徑讀取工具也引進來

import tensorflow as tf # 就是 Tensorflow 不用多說
import tensorflow_hub as hub # Tensorflow bert 社群
import tensorflow_text as text # 前處理
from official.nlp import optimization  # 優化工具

import matplotlib.pyplot as plt # 畫圖用的套件

tf.get_logger().setLevel('ERROR') # Debug的工具，也可以不寫

2025-01-13 00:01:47.566877: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-01-13 00:01:47.566938: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/ysh/miniconda3/envs/3.9/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/ysh/miniconda3/envs/3.9/lib/python3.9/site-packages/tensorflow_addons/u

In [2]:
# # 這個是資料集的下載連結，如果你複製這段 url 到瀏覽器上，也可以下載的到
# url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

# dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
#                                   untar=True, cache_dir='.',
#                                   cache_subdir='')

# # 所謂的 os.path.join 就是把兩個路徑結合起來的意思，如果路徑是 ./data/，join之後就會變成 ./data/aclImdb，而以下所做的，就是要整合資料路徑，方便之後模型讀取
# dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

# train_dir = os.path.join(dataset_dir, 'train')

# # remove unused folders to make it easier to load the data
# remove_dir = os.path.join(train_dir, 'unsup')
# shutil.rmtree(remove_dir)


In [38]:
from core.general import *
ls('../data')

import pandas as pd
f = pd.read_csv('../data/train_data.csv')
f = f[['Statement', 'tags']]
f['isDP'] = ['dp' in i for i in f['tags'].tolist()]
f = f.drop('tags', axis = 1)
f

,Statement,isDP
0,After watching the new over-rated series Squid...,False
1,Today on a math lesson the teacher told Vovoch...,False
2,"A sequence of integers $$$b_1, b_2, \ldots, b_...",True
3,Petya got interested in grammar on his third y...,False
4,"Nezzar designs a brand new game ""Hidden Permut...",False
...,...,...
7971,"You are given an array of integers $$$a_1,a_2,...",True
7972,Polycarp has a poor memory. Each day he can re...,False
7973,This is the hard version of the problem. The o...,False
7974,"Two chess pieces, a rook and a knight, stand o...",False


In [47]:
pre = f.to_dict()
# for i in train:
# debug(pwd())
cd('aclImdb/train')
for tag in ['pos', True], ['neg', False]:
    cd(tag[0])
    for i in range(len(f['Statement'])):
        if pre['isDP'][i] ^ tag[1]: continue;
        write_to_file(f'{i}.txt', pre['Statement'][i])
    cd('..')
cd('../..')

FileNotFoundError: [Errno 2] No such file or directory: 'aclImdb/train'

In [5]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test',
    batch_size=batch_size)

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [6]:
for text_batch, label_batch in train_ds.take(1):
    print(f'Review: {text_batch.numpy()[0]}')
    for i in range(3):
        label = label_batch.numpy()[i]
        print(f'Label : {label} ({class_names[label]})')

Review: b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
Label : 0 (neg)
Label : 0 (neg)
Label : 1 (pos)


2025-01-13 00:04:06.407146: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
